#### What Require Inputs in this Notebook
1. [Fill in File Input](#Load-in-Data)
2. [Right or Left Hand](#Right-or-Left-Hand)
3. [Output ShotCSV](#Output-ShotCSV)
4. [Change to atNet (by Player) from tagger later](#At-Net)
5. [EDA and get Stats Here](#EDA)
6. [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
7. [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)

# Table of Contents
1. [Load in Data](#Load-in-Data)
- Error Checking
- Add Columns
- [Output ShotCSV](#Output-ShotCSV)
2. [Create Point DF](#Create-Point-DF)
- Add Columns
- [Output PointCSV for Visuals](#PRINT-POINTCSV-VISUALS)
- [Output PointCSV for Upload](#Print-POINTCSV-UPLOAD)    
        - cut out points with no timestamp position    
        - atNetPlayer1 has values of the player name for display purposes instead of boolean values

# [Click for Summary Stats and EDA HERE](#EDA)

### List of Changes

For Shot CSV
- Adds Point Number (Comment out if already there)
- Changes ShotType to ShotDirection and ShotFhBh (old tagger)
- Forward Fills these columns'gameScore','setScore', 'tiebreakScore', 'serverName', 'player1Name', 'player2Name'

Columns Added:
  - 'isDoubleFault',
  - 'pointWonBy',
  - 'lastShotError',
  - 'serveResult',
  - 'isInsideIn',
  - 'returnerName',
  - 'serveInPlacement',
  - 'player1Hand',
  - 'player2Hand',
  - 'shotHitBy',
  - 'isInsideOut'
  

For Point (Visuals)
- point related info, and player1 focused stats


For Point (Upload)
- removes columns with no start time
- changes "1" value of atNetPlayer1 to player1Name

### Warning: Changes to be made soon
- atNetPlayer1, atNetPlayer2 are being aggregated by if hitting a volley. Will be replaced by tagger values for those
- pointWonBy is being calculated, replaced by tagger soon
- player1Hand and player2Hand are being manually inputted

### Future Changes to Notebook
- More stats on approach shots
- Pressure Point stats, like what did players did on 30-30 points onwards
- Inside Out/Inside In Stats once we get matches with accurate side info
- handling of rows where there is error on video side and the point is cut short

Notes:
- Keep empty values None Type, they will be converted to empty string "" right before outputting to Point CSV Only for Upload
- Having 1 does not mean other value is boolean 0, it is None

# Notebook Start

### Load in Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Put your file name here

your_file_name = "Cleaned Tian_Fangran_OhioState_Full_Match - Tian_Fangran_OhioState_Full_Match.csv"
shot_data = pd.read_csv(your_file_name)



In [3]:
# Check existing columns

shot_data.columns

Index(['pointScore', 'gameScore', 'setScore', 'tiebreakScore', 'isPointStart',
       'pointStartTime', 'isPointEnd', 'pointEndTime', 'pointEndTime.1',
       'isBreakPoint', 'shotInRally', 'side', 'serverName', 'firstServeIn',
       'firstServeZone', 'firstServeXCoord', 'firstServeYCoord',
       'secondServeIn', 'secondServeZone', 'secondServeXCoord',
       'secondServeYCoord', 'isAce', 'returnContactX', 'returnContactY',
       'returnPlacementX', 'returnPlacementY', 'shotContactX', 'shotContactY',
       'shotDirection', 'shotFhBh', 'isSlice', 'isVolley', 'isOverhead',
       'isApproach', 'isDropshot', 'atNetPlayer1', 'atNetPlayer2', 'isLob',
       'shotLocationX', 'shotLocationY', 'isWinner', 'isErrorWideR',
       'isErrorWideL', 'isErrorNet', 'isErrorLong', 'clientTeam', 'Date',
       'Division', 'Event', 'lineupPosition', 'matchDetails', 'opponentTeam',
       'player1Name', 'player2Name', 'Round', 'Surface', 'Notes'],
      dtype='object')

<!-- ### Error Check: 
#### If data includes firstServeZone but no "1" for firstServeIn, same for secondServeZone/secondServeIn -->

### Error Check: 
#### If returnData exists, separate values of "Forehand/Backhand Crosscourt/Down the Line" into shotFhBh and shotDirection

In [4]:
# Check if 'returnData' column exists
if 'returnData' in shot_data.columns:
    # Replace NaN values in returnData column with an empty string
    shot_data['returnData'].fillna('', inplace=True)

    # Check if "backhand" or "forehand" is in returnData and update shotFhBh accordingly
    shot_data.loc[shot_data['returnData'].str.contains('backhand', case=False), 'shotFhBh'] = 'Backhand'
    shot_data.loc[shot_data['returnData'].str.contains('forehand', case=False), 'shotFhBh'] = 'Forehand'

    # Check if "Crosscourt" or "Down the Line" is in returnData and update shotDirection accordingly
    shot_data.loc[shot_data['returnData'].str.contains('Crosscourt', case=False), 'shotDirection'] = 'Crosscourt'
    shot_data.loc[shot_data['returnData'].str.contains('Down the Line', case=False), 'shotDirection'] = 'Down the Line'

    # Display the modified shot_data
    print(shot_data[['returnData', 'shotDirection', 'shotFhBh']].head(10))
else:
    print("Column 'returnData' does not exist.")


Column 'returnData' does not exist.


### Error Check: 
#### Change ShotType to shotFhBh if old tagger

In [5]:
if 'shotType' in shot_data.columns:
    # Rename the column
    shot_data.rename(columns={'shotType': 'shotFhBh'}, inplace=True)
    print("Column renamed successfully.")
else:
    print("Check Passed: Column 'shotType' does not exist.")

Check Passed: Column 'shotType' does not exist.


### Error Check: 
#### From Old Tagger, checks for shotFhBh doesn't include direction and fh/bh

In [6]:
shot_data.loc[shot_data['shotFhBh'] == 'Forehand Crosscourt', 'shotDirection'] = 'Crosscourt'
shot_data.loc[shot_data['shotFhBh'] == 'Forehand Crosscourt', 'shotFhBh'] = 'Forehand'

shot_data.loc[shot_data['shotFhBh'] == 'Backhand Crosscourt', 'shotDirection'] = 'Crosscourt'
shot_data.loc[shot_data['shotFhBh'] == 'Backhand Crosscourt', 'shotFhBh'] = 'Backhand'

shot_data.loc[shot_data['shotFhBh'] == 'Forehand Down the Line', 'shotDirection'] = 'Down the Line'
shot_data.loc[shot_data['shotFhBh'] == 'Forehand Down the Line', 'shotFhBh'] = 'Forehand'

shot_data.loc[shot_data['shotFhBh'] == 'Backhand Down the Line', 'shotDirection'] = 'Down the Line'
shot_data.loc[shot_data['shotFhBh'] == 'Backhand Down the Line', 'shotFhBh'] = 'Backhand'

print(shot_data[['shotDirection', 'shotFhBh']].head(10))

   shotDirection  shotFhBh
0            NaN       NaN
1  Down the Line  Backhand
2     Crosscourt  Forehand
3     Crosscourt  Forehand
4     Crosscourt  Forehand
5            NaN       NaN
6            NaN       NaN
7     Crosscourt  Backhand
8  Down the Line  Forehand
9  Down the Line  Backhand


### Error Check: 
#### Checks Player1Name and Player2Name have Values

In [7]:
fillInPlayer1Name = "Choose_Player1Name_Here"
fillInPlayer2Name = "Choose_Player2Name_Here"

if shot_data.at[0, 'player1Name'] == None:
    print("player1Name was blank. Filling with 'fillInplayer1Name'")
    shot_data.at[0, 'player1Name'] = fillInPlayer1Name
else:
    print("Check Passed: player1Name is not blank. Current value:", shot_data.at[0, 'player1Name'])

# Check if player2Name is blank, if so, fill it with 'fillInplayer2Name'
if shot_data.at[0, 'player2Name'] == None:
    print("player2Name was blank. Filling with 'fillInplayer2Name'")
    shot_data.at[0, 'player2Name'] = fillInPlayer2Name
else:
    print("Check Passed: player2Name is not blank. Current value:", shot_data.at[0, 'player2Name'])


Check Passed: player1Name is not blank. Current value: Tian Fangran
Check Passed: player2Name is not blank. Current value: Irina Cantos Siemers


In [8]:
### Error Check: 
#### Fills in PointNumber if not there

In [9]:
# Filter out rows with 'side' values other than 'Deuce' and 'Ad'
filtered_data = shot_data[shot_data['side'].isin(['Deuce', 'Ad'])]

# Rows that don't meet the condition
rows_not_meeting_condition = shot_data[~shot_data['side'].isin(['Deuce', 'Ad'])]

# Count of rows not meeting the condition
count_rows_not_meeting_condition = len(rows_not_meeting_condition)
print("Check Passed!\n")

print("Rows with Side that aren't Deuce or Ad:", count_rows_not_meeting_condition)

# Output rows if you want to see which rows
# print("Rows not meeting the condition:")
# print(rows_not_meeting_condition)


Check Passed!

Rows with Side that aren't Deuce or Ad: 0


### Error Check: 
#### Fills in PointNumber if not there

In [10]:
 # Check if both conditions passed
if 'pointNumber' in shot_data.columns and not shot_data['pointNumber'].isnull().any() and shot_data['pointNumber'].is_monotonic_increasing:
    print("Check Passed: Point Numbers already exist")


# Check if pointNumber exists for every row and is not empty
if 'pointNumber' not in shot_data.columns or shot_data['pointNumber'].isnull().any():
    # Execute the script to calculate pointNumber
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
    
    print("Data had missing point numbers. They were filled in automatically")
    
# Check if pointNumber is in increasing order
if not shot_data['pointNumber'].is_monotonic_increasing:
    # Print the condition failure if the 'pointNumber' column is not in increasing order
    print("Data had point numbers in the wrong order. They were filled in automatically")
    point_starts = (shot_data['isPointStart'] == 1)
    shot_data['pointNumber'] = point_starts.cumsum()
   

Data had missing point numbers. They were filled in automatically


In [11]:
# print(shot_data.tail())

In [12]:
missing_pointNumber_rows = shot_data[shot_data['pointNumber'].isnull()]

# Check if there are missing rows
if len(missing_pointNumber_rows) == 0:
    print("Check Passed: All rows have pointNumber")
else:
    print(f"Count of rows missing 'pointNumber': {len(missing_pointNumber_rows)}")

Check Passed: All rows have pointNumber


### Add Column: player1Hand, player2Hand

In [44]:
fillInplayer1Hand = "Right"
fillInplayer2Hand = "Right"

# Check if player1Hand key exists in shot_data, if not, create it
if 'player1Hand' not in shot_data:
    shot_data['player1Hand'] = None

# Check if player2Hand key exists in shot_data, if not, create it
if 'player2Hand' not in shot_data:
    shot_data['player2Hand'] = None

if shot_data.at[0, 'player1Hand'] == None:
    print("player1Hand was blank. Filling with 'fillInplayer1Hand'")
    shot_data.at[0, 'player1Hand'] = fillInplayer1Hand
else:
    print("Check Passed: player1Hand is not blank. Current value:", shot_data.at[0, 'player1Hand'])

# Check if player2Hand is blank, if so, fill it with 'fillInplayer2Hand'
if shot_data.at[0, 'player2Hand'] == None:
    print("player2Hand was blank. Filling with 'fillInplayer2Hand'")
    shot_data.at[0, 'player2Hand'] = fillInplayer2Hand
else:
    print("Check Passed: player2Hand is not blank. Current value:", shot_data.at[0, 'player2Hand'])


print("Player1Hand Current value:", shot_data.at[0, 'player1Hand'])
print("Player2Hand Current value:", shot_data.at[0, 'player2Hand'])

Check Passed: player1Hand is not blank. Current value: Right
Check Passed: player2Hand is not blank. Current value: Right
Player1Hand Current value: Right
Player2Hand Current value: Right


### Filling Columns: Forward Fill
- 'gameScore'
- 'setScore'
- 'serverName'
- 'player1Name',
- 'player2Name'
- 'player2Hand'

# Forward Fill GameScore, SetScore, tiebreakScore, serverName, player1Name, player2Name

In [45]:
columns_to_fill = ['gameScore','setScore', 'serverName', 'player1Name', 'player2Name', 'player1Hand', 'player2Hand'] 
# columns_to_fill = ['gameScore','setScore', 'tiebreakScore', 'serverName', 'player1Name', 'player2Name'] 
# Add player1Hand, player2Hand when it is in tagger

for column in columns_to_fill:
    shot_data[column].replace(['', 'na'], pd.NaT, inplace=True)
    shot_data[column] = shot_data[column].ffill()

### Error Check: 
#### Check for NA's

- all 0, except missing pointScore should match amount of tiebreak shots
- side can have missing, for old tagger and not tagging side of each shot

### Fixing:
- Open CSV in Google Sheets (Excel will change to date format), output game, set, point Score of missing values. Then find in google sheets, adjust, redownload, and re-upload into notebook

In [46]:
# Count empty strings in each column
empty_string_counts = (shot_data == "").sum()

# Filter out columns with zero empty strings
non_zero_counts = empty_string_counts[empty_string_counts > 0]

# Count NaN values in 'pointScore' column when 'gameScore' is '6-6'
na_tiebreak_count = shot_data.loc[shot_data['gameScore'] == '6-6', 'pointScore'].isna().sum()

side_na_count = shot_data.loc[shot_data['isPointStart'] == 1, 'side'].isna().sum()
print("\nCount of NaN values for 'side' on Point Start:", side_na_count)

# Display the count
print("\nCount of Nan when gameScore is '6-6' (# tiebreak shots):", na_tiebreak_count)


# Count NaN or empty values in specified columns
na_counts = shot_data[[ 'pointScore', 'shotInRally', 'gameScore', 'setScore', 'side', 'serverName']].isna().sum()

# Display the counts
print(f"\nCout of NA's in these columns\n{na_counts}")

# Display the counts
print("Count of empty strings in each column that includes at least one:")
print(non_zero_counts)




Count of NaN values for 'side' on Point Start: 0

Count of Nan when gameScore is '6-6' (# tiebreak shots): 0

Cout of NA's in these columns
pointScore     0
shotInRally    0
gameScore      0
setScore       0
side           0
serverName     0
dtype: int64
Count of empty strings in each column that includes at least one:
Series([], dtype: int64)


### Error Check: 
####  rows where side is empty
- If side missing, focus on rows where (shotInRally == 1) and returns (shotInRally == 2 ), anything else can leave NA

In [47]:
print(shot_data[shot_data["side"].isnull()][['pointScore', 'gameScore', 'setScore','side', 'shotInRally']])

Empty DataFrame
Columns: [pointScore, gameScore, setScore, side, shotInRally]
Index: []


### Error Check: 
####  Outputs Missing shotInRally Rows

In [48]:
empty_shot_rows = shot_data[shot_data['shotInRally'].isnull()]

if not empty_shot_rows.empty:
    # Iterate over the index of empty_shot_rows
    for index in empty_shot_rows.index:
        # Get the row with empty shotInRally
        empty_row = shot_data.loc[index]
        
        # Get the row above it
        if index - 1 >= 0:
            above_row = shot_data.loc[index - 1]
        else:
            above_row = None
        
        # Get the row below it
        if index + 1 < len(shot_data):
            below_row = shot_data.loc[index + 1]
        else:
            below_row = None
        
        # Print the rows
        print("Empty row:")
        print(empty_row)
        print("Row above:")
        print(above_row)
        print("Row below:")
        print(below_row)
        print("====================")
else:
    print("Check Passed: No Missing shotInRally")


Check Passed: No Missing shotInRally


### Error Check: 
####  Check that pointNumber is increasing consecutively

- if video is bad and points are cut out, this can fail and that's ok.

In [49]:
point_numbers = shot_data['pointNumber'].unique()

# Check if the point numbers are consecutive
if list(point_numbers) == list(range(1, len(point_numbers) + 1)):
    print("Check Passed")
else:
    print("Error: The 'pointNumber' column does not contain consecutive numbers starting from 1.")


Check Passed


### Error Check: 
#### Check same amount of start and end points

In [50]:
# Count of Point Start and Point End
num_point_start = shot_data['isPointStart'].sum()

# Count the number of rows where isPointEnd is equal to 1
num_point_end = shot_data['isPointEnd'].sum()

print("Number of rows with isPointStart = 1:", num_point_start)
print("Number of rows with isPointEnd = 1:", num_point_end)

Number of rows with isPointStart = 1: 107.0
Number of rows with isPointEnd = 1: 107.0


### Error Check: 
#### Output consecutive isPointStart with no isPointEnd, and vice versa for isPointEnd
(Sometimes Doesn't Work"

In [51]:
# Output total start, total end
# How many Points aren't enclosing between point start and point end 

# Count where isPointStart = 1
count_isPointStart = (shot_data['isPointStart'] == 1).sum()

# Count where isPointEnd = 1
count_isPointEnd = (shot_data['isPointEnd'] == 1).sum()

print("Total count where isPointStart = 1:", count_isPointStart)
print("Total count where isPointEnd = 1:", count_isPointEnd)



# Initialize variables to keep track of the state
has_start = False
overlapping_ends = []

# Iterate through the DataFrame
for i in range(len(shot_data)):
    if shot_data['isPointStart'][i] == 1:
        has_start = True
    elif shot_data['isPointEnd'][i] == 1:
        if not has_start:
            overlapping_ends.append(i)
        else:
            has_start = False

# Display the rows where isPointEnd doesn't have a corresponding isPointStart
print("Rows where isPointEnd doesn't have a corresponding isPointStart:")
print(shot_data.iloc[overlapping_ends])

# Initialize variables to keep track of the state
start_indices = []
consecutive_starts = []

# Iterate through the DataFrame
for i in range(len(shot_data) - 1):  # Iterate until the second last row
    if shot_data['isPointStart'][i] == 1 and shot_data['isPointEnd'][i] != 1:
        # Check if the next row also has isPointStart without a corresponding isPointEnd
        if shot_data['isPointStart'][i + 1] == 1 and shot_data['isPointEnd'][i + 1] != 1:
            consecutive_starts.append(i + 1)

# Display the rows where isPointStart may double up without a corresponding isPointEnd
print("Rows where isPointStart may double up without a corresponding isPointEnd:")
print(shot_data.iloc[consecutive_starts])


Total count where isPointStart = 1: 107
Total count where isPointEnd = 1: 107
Rows where isPointEnd doesn't have a corresponding isPointStart:
Empty DataFrame
Columns: [pointScore, gameScore, setScore, tiebreakScore, isPointStart, pointStartTime, isPointEnd, pointEndTime, pointEndTime.1, isBreakPoint, shotInRally, side, serverName, firstServeIn, firstServeZone, firstServeXCoord, firstServeYCoord, secondServeIn, secondServeZone, secondServeXCoord, secondServeYCoord, isAce, returnContactX, returnContactY, returnPlacementX, returnPlacementY, shotContactX, shotContactY, shotDirection, shotFhBh, isSlice, isVolley, isOverhead, isApproach, isDropshot, atNetPlayer1, atNetPlayer2, isLob, shotLocationX, shotLocationY, isWinner, isErrorWideR, isErrorWideL, isErrorNet, isErrorLong, clientTeam, Date, Division, Event, lineupPosition, matchDetails, opponentTeam, player1Name, player2Name, Round, Surface, Notes, pointNumber, player1Hand, player2Hand, returnerName, shotHitBy, isInsideOut, isInsideIn]
In

### Error Check: 
#### Make Jan-00 back into 1-0 for Game/Set Score

- In case you open in Excel and it changes score format to dates

In [52]:
# Make Scores Strings not Date Time
columns_to_convert = ['gameScore', 'setScore'] #if no tiebreakScore
# columns_to_convert = ['gameScore', 'setScore', 'tiebreakScore']
shot_data[columns_to_convert] = shot_data[columns_to_convert].astype(object)

In [53]:
import re

# Define a mapping for month abbreviations
month_mapping = {'Jan': '1', 'Feb': '2', 'Mar': '3', 'Apr': '4', 'May': '5', 'Jun': '6',
                 'Jul': '7', 'Aug': '8', 'Sep': '9', 'Oct': '10', 'Nov': '11', 'Dec': '12'}

# Function to convert string like 'Jan-00' to '1-0'
def convert_score_string(score_str):
    # Check if the string has a month abbreviation and a year ending with '00'
    if re.match(r'^\d{1,2}-[A-Za-z]{3}$', score_str):
        # Extract year and month abbreviation
        year, month = score_str.split('-')

        # Remove leading zeros from the year
        year = str(int(year))

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Concatenate the parts to form the transformed string
        transformed_str = f'{year}-{month_number}'

        return transformed_str

    # Check if the string has a month abbreviation and a year with leading '0's
    elif re.match(r'^[A-Za-z]{3}-\d{1,2}$', score_str):
        # Extract month abbreviation and year
        month, year = score_str.split('-')

        # Replace month abbreviation with corresponding number
        month_number = month_mapping.get(month, month)

        # Remove leading zeros from the year
        year = str(int(year))

        # Concatenate the parts to form the transformed string
        transformed_str = f'{month_number}-{year}'

        return transformed_str

    return score_str

# Apply the conversion function to the relevant columns in shot_data
shot_data['gameScore'] = shot_data['gameScore'].apply(convert_score_string)
shot_data['setScore'] = shot_data['setScore'].apply(convert_score_string)
# shot_data['tiebreakerScore'] = shot_data['tiebreakerScore'].apply(convert_score_string)


### Error Check: 
#### Check that there is no strings in set, game scores. 

In [54]:
# Assuming shot_data is your DataFrame
unique_set_scores = shot_data['setScore'].unique()
unique_game_scores = shot_data['gameScore'].unique()

# Print unique values
print("Unique Set Scores:", unique_set_scores)
print("Unique Game Scores:", unique_game_scores)


Unique Set Scores: ['0-0' '0-1']
Unique Game Scores: ['0-0' '1-0' '1-1' '2-1' '2-2' '2-3' '3-3' '3-4' '3-5' '1-2' '1-3' '1-4'
 '2-4']


# Add More Columns

### Add Column: returnerName

In [55]:
# Get Returner Name for Shot DF

# Extract player1Name and player2Name from the first row of shot_data
first_row_shot_df = shot_data.iloc[0]
player1_name = first_row_shot_df['player1Name']
player2_name = first_row_shot_df['player2Name']

# # Can Also Manually inpute Player 1 and 2 name
# player1_name = "Kimmi Hance"
# player2_name = "Malaika Rapolu"

def get_returner_name(server_name):
    return player2_name if server_name == player1_name else player1_name

# Add 'Returner Name' column to point_df using the function
shot_data['returnerName'] = shot_data['serverName'].apply(get_returner_name)

print(f"Player 1 = {player1_name}, Player 2 = {player2_name}")
# print(shot_data[['serverName','returnerName']])

Player 1 = Tian Fangran, Player 2 = Irina Cantos Siemers


### Add Column: shotHitBy

In [56]:
# Add Shot Hit By for each Shot

shot_data['shotHitBy'] = shot_data.apply(lambda row: row['serverName'] if row['shotInRally'] % 2 == 1 else row['returnerName'], axis=1)
# print(shot_data[['serverName', 'shotHitBy','pointNumber', 'shotInRally']].head(20))

### Add Column: isInsideOut, isInsideIn

In [58]:
# Add columns for isInsideOut and isInsideIn, initially set to 0
shot_data['isInsideOut'] = None
shot_data['isInsideIn'] = None

# Iterate through rows
for index, row in shot_data.iterrows():
    shotHitBy = row['shotHitBy']
    player_hand = row['player1Hand'] if shotHitBy == row['player1Name'] else row['player2Hand']
    

    
    if player_hand == "Right":
        if row['side'] == "Deuce" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
    elif player_hand == "Left":
        if row['side'] == "Ad" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Crosscourt":
            shot_data.at[index, 'isInsideOut'] = 1
        elif row['side'] == "Ad" and row['shotFhBh'] == "Backhand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1
        elif row['side'] == "Deuce" and row['shotFhBh'] == "Forehand" and row['shotDirection'] == "Down the Line":
            shot_data.at[index, 'isInsideIn'] = 1

### Add Column: isAce

In [59]:
# Add the Ace column
shot_data['isAce'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] == 1 or row['secondServeIn'] == 1): # either first or second serve went in
                shot_data.at[index, 'isAce'] = 1

### Add Column: isDoubleFault

In [60]:
# Add the DoubleFault column
shot_data['isDoubleFault'] = None

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if (row['firstServeIn'] != 1 and row['secondServeIn'] != 1): # either first or second serve went in
                shot_data.at[index, 'isDoubleFault'] = 1

### Add Column: pointWonBy, lastShotError

In [61]:
# Add the 'pointWonBy' column
shot_data['pointWonBy'] = None

# Add the 'lastShotError' column
shot_data['lastShotError'] = 0

for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        if row['shotInRally'] == 1: # last point is serve
            if row['isAce'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['serverName']
            elif row['isDoubleFault'] == 1: 
                shot_data.at[index, 'pointWonBy'] = row['returnerName']

                
        elif row['shotInRally'] != 1:
            if row['isErrorWideR'] == 1 or row['isErrorWideL'] == 1 or row['isErrorNet'] == 1 or row['isErrorLong'] == 1: # if error
                shot_data.at[index, 'lastShotError'] = 1
                
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
        
            elif row['isWinner'] == 1:
                if row['shotInRally'] % 2 == 0:
                    shot_data.at[index, 'pointWonBy'] = row['returnerName']
                else:
                    shot_data.at[index, 'pointWonBy'] = row['serverName']


### Error Check: 
#### Output Point End with no pointWonBy


In [62]:
# Missing Point End
print(shot_data[(shot_data['isPointEnd'] == 1) & (shot_data['pointWonBy'].isnull())][['pointScore', 'gameScore','setScore', 'lastShotError', 'isWinner', 'isErrorWideR', 'isErrorWideL',
       'isErrorNet', 'isErrorLong', 'pointWonBy', 'serverName', 'shotInRally']])


Empty DataFrame
Columns: [pointScore, gameScore, setScore, lastShotError, isWinner, isErrorWideR, isErrorWideL, isErrorNet, isErrorLong, pointWonBy, serverName, shotInRally]
Index: []


### Backward Fill: PointWonBy

In [63]:
shot_data['pointWonBy'].bfill()

0              Tian Fangran
1              Tian Fangran
2              Tian Fangran
3              Tian Fangran
4              Tian Fangran
               ...         
579    Irina Cantos Siemers
580    Irina Cantos Siemers
581    Irina Cantos Siemers
582    Irina Cantos Siemers
583    Irina Cantos Siemers
Name: pointWonBy, Length: 584, dtype: object

### Add Column: serveResult, serveInPlacement

In [64]:
conditions = [
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] == 1),
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] != 1) & (shot_data['secondServeIn'] == 1),
    (shot_data['isPointStart'] == 1) & (shot_data['firstServeIn'] != 1) & (shot_data['secondServeIn'] != 1)
]

# Define the values to be assigned for each condition
values_result = ['1st Serve In', '2nd Serve In', 'Double Fault']
values_placement = [shot_data['firstServeZone'], shot_data['secondServeZone'], np.nan]

# Use numpy.select to assign values based on conditions
shot_data['serveResult'] = np.select(conditions, values_result, default='')
shot_data['serveInPlacement'] = np.select(conditions, values_placement, default='')

# Now shot_data DataFrame is updated with serveResult and serveInPlacement values where isPointStart is 1
# print(shot_data[['serveResult', 'serveInPlacement','pointScore','side']].head(60))

In [65]:
shot_data.replace("", None, inplace=True)

# Double Check BEFORE Outputting

In [66]:
print(shot_data.head(10))

  pointScore gameScore setScore  tiebreakScore  isPointStart  pointStartTime  \
0        0-0       0-0      0-0            NaN           1.0          1663.0   
1        0-0       0-0      0-0            NaN           NaN             NaN   
2        0-0       0-0      0-0            NaN           NaN             NaN   
3        0-0       0-0      0-0            NaN           NaN             NaN   
4        0-0       0-0      0-0            NaN           NaN             NaN   
5       0-15       0-0      0-0            NaN           1.0         29046.0   
6       0-30       0-0      0-0            NaN           1.0         72945.0   
7       0-30       0-0      0-0            NaN           NaN             NaN   
8       0-30       0-0      0-0            NaN           NaN             NaN   
9       0-30       0-0      0-0            NaN           NaN             NaN   

   isPointEnd  pointEndTime  pointEndTime.1  isBreakPoint  ...  player2Hand  \
0         NaN           NaN             

In [67]:
print(shot_data.tail(10))

    pointScore gameScore setScore  tiebreakScore  isPointStart  \
574      15-30       3-5      0-1            NaN           NaN   
575      15-40       3-5      0-1            NaN           1.0   
576      15-40       3-5      0-1            NaN           NaN   
577      15-40       3-5      0-1            NaN           NaN   
578      15-40       3-5      0-1            NaN           NaN   
579      15-40       3-5      0-1            NaN           NaN   
580      15-40       3-5      0-1            NaN           NaN   
581      15-40       3-5      0-1            NaN           NaN   
582      15-40       3-5      0-1            NaN           NaN   
583      15-40       3-5      0-1            NaN           NaN   

     pointStartTime  isPointEnd  pointEndTime  pointEndTime.1  isBreakPoint  \
574             NaN         1.0           NaN             NaN           NaN   
575             NaN         NaN           NaN             NaN           1.0   
576             NaN         NaN     

# Output ShotCSV

In [68]:
# # # Ouput Improved Shot Csv HERE
# # Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = shot_data.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = shot_data.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
shot_data.to_csv(f'Shot_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)

In [69]:
shot_data['pointNumber'].unique()

array([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
        14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
        27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
        40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
        53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
        66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
        79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
        92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
       105, 106, 107])

# Below is for Point CSV

### Create Point DF

In [70]:
# Creating point_df (with only 1 row for each pointNumber)
point_df = shot_data.drop_duplicates(subset='pointNumber')[['pointNumber']]
# point_df.shape

### Add Column: player1Name, player2Name
### Set Variable: first_player1Name, first_player2Name

In [71]:
# Extract the first value of player1Name and player2Name from shot_data
first_player1Name = shot_data['player1Name'].iloc[0]
first_player2Name = shot_data['player2Name'].iloc[0]

# Fill in the first value into all rows of point_df['player1Name'] and point_df['player2Name']
point_df['player1Name'] = first_player1Name
point_df['player2Name'] = first_player2Name

### Add Column: Scores

In [72]:
point_df['pointScore'] = shot_data.groupby('pointNumber')['pointScore'].first().values
point_df['gameScore'] = shot_data.groupby('pointNumber')['gameScore'].first().values
point_df['setScore'] = shot_data.groupby('pointNumber')['setScore'].first().values
point_df['tiebreakScore'] = shot_data.groupby('pointNumber')['tiebreakScore'].first().values

### Add Column: Side

In [73]:
# Group shot_data by 'pointNumber' and get the first 'side' value for each group
side_values = shot_data.groupby('pointNumber')['side'].first().reset_index()

point_df['side'] = side_values['side'].values

### Add Column: serverName, returnerName

In [74]:
# Adds Server and Returner Names and pointScore

point_df['serverName'] = shot_data.groupby('pointNumber')['serverName'].first().values
point_df['returnerName'] = shot_data.groupby('pointNumber')['returnerName'].first().values


In [75]:
client_team_value = shot_data.loc[0, 'clientTeam']
opponent_team_value = shot_data.loc[0, 'opponentTeam']

# Adding 'clientTeam' and 'opponentTeam' columns to point_df
point_df['clientTeam'] = client_team_value
point_df['opponentTeam'] = opponent_team_value


## Warning: Will be Empty if Timestamp separate
### Add Column: serverName, returnerName

In [76]:
# Add Start and End times per point

for index, row in shot_data.iterrows():
    point_number = row['pointNumber']
    
    if row['isPointStart'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'Position'] = row['pointStartTime']
    if row['isPointEnd'] == 1:
        point_df.loc[point_df['pointNumber'] == point_number, 'pointEndPosition'] = row['pointEndTime']

# Add Duration
point_df['Duration'] = point_df['pointEndPosition'] - point_df['Position']

### Add Column: Rally Column
- rallyCount (x amount)
- rallyCountFreq (1-4,5-8,9-12,13+)

In [77]:
# Find the highest shotInRally for each pointNumber in shot_data
max_rally_per_point = shot_data.groupby('pointNumber')['shotInRally'].max().reset_index()
point_df['rallyCount'] = list(max_rally_per_point['shotInRally'])

# Add 'rallyCountFreq' column based on specified conditions
point_df['rallyCountFreq'] = point_df['rallyCount'].apply(lambda x: '1 - 4' if 1 <= x <= 4 else ('5 - 8' if 5 <= x <= 8 else ('9 - 12' if 9 <= x <= 12 else ('13 +' if x >= 13 else 'Error'))))


### Add Column: Serve 
- firstServeIn
- secondServeIn
- serveResult
- serveInPlacement

Part 2
- serveResult
- serveInPlacement
 
Part 3
- firstServeZone
- secondServeZone

In [78]:
# Add firstServeIn and secondServeIn

# Add firstServeIn and secondServeIn columns
point_df['firstServeIn'] = 0
point_df['secondServeIn'] = 0

for point_number in shot_data['pointNumber'].unique():
    # Check if firstServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['firstServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'firstServeIn'] = 1
    
    # Check if secondServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['secondServeIn'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'secondServeIn'] = 1

#part 2
start_points = shot_data[shot_data['isPointStart'] == 1]

# Set values in point_df to corresponding values from start_points
point_df['serveResult'] = start_points['serveResult'].values
point_df['serveInPlacement'] = start_points['serveInPlacement'].values

# part 3
serve_zones = shot_data.loc[shot_data['shotInRally'] == 1, ['pointNumber', 'firstServeZone', 'secondServeZone', 'firstServeIn', 'secondServeIn']].drop_duplicates()
point_df['firstServeZone'] = shot_data.groupby('pointNumber')['firstServeZone'].first().values
point_df['secondServeZone'] = shot_data.groupby('pointNumber')['secondServeZone'].first().values

### Add Column: Ace

In [79]:
point_df['isAce'] = ((point_df['rallyCount'] == 1) & ((point_df['serveResult'] != "Double Fault")))

# Display the resulting DataFrame
print(point_df[['pointNumber', 'rallyCount', 'isAce']].head(14))

    pointNumber  rallyCount  isAce
0             1           5  False
5             2           1  False
6             3           5  False
11            4           4  False
15            5           2  False
17            6           3  False
20            7          10  False
30            8           1  False
31            9           2  False
33           10           6  False
39           11           4  False
43           12           5  False
48           13           9  False
57           14           2  False


### Add Column: Return

In [80]:
# print(shot_data[shot_data['shotInRally'] == 2][['shotDirection','shotFhBh','pointScore','gameScore','setScore']])

In [81]:
# Set the initial values of 'returnDirection' and 'returnHand' columns to None
point_df['returnDirection'] = None
point_df['returnFhBh'] = None

# Iterate through pointNumber in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if shotInRally == 2 exists for the given pointNumber
    if 2 in shot_data.loc[shot_data['pointNumber'] == point_number, 'shotInRally'].values:
        # Get the information from the corresponding row
        row_with_return_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['shotInRally'] == 2)].iloc[0]

        # Assign values to 'returnDirection' and 'returnHand' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'returnDirection'] = row_with_return_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'returnFhBh'] = row_with_return_info['shotFhBh']

# Display the modified DataFrame
print(point_df.head(10))

    pointNumber   player1Name           player2Name pointScore gameScore  \
0             1  Tian Fangran  Irina Cantos Siemers        0-0       0-0   
5             2  Tian Fangran  Irina Cantos Siemers       0-15       0-0   
6             3  Tian Fangran  Irina Cantos Siemers       0-30       0-0   
11            4  Tian Fangran  Irina Cantos Siemers      15-30       0-0   
15            5  Tian Fangran  Irina Cantos Siemers      30-30       0-0   
17            6  Tian Fangran  Irina Cantos Siemers      40-30       0-0   
20            7  Tian Fangran  Irina Cantos Siemers      40-40       0-0   
30            8  Tian Fangran  Irina Cantos Siemers        0-0       1-0   
31            9  Tian Fangran  Irina Cantos Siemers       0-15       1-0   
33           10  Tian Fangran  Irina Cantos Siemers       0-30       1-0   

   setScore  tiebreakScore   side            serverName          returnerName  \
0       0-0            NaN  Deuce  Irina Cantos Siemers          Tian Fangran   
5

### Add Column: Error Column
- errorType

In [82]:
# Create an empty DataFrame to store the results
error_results = pd.DataFrame(columns=['errorType', 'pointNumber'])

# Iterate through entire shot_data
for index, row in shot_data.iterrows():
    pointNumber = row['pointNumber']
    point_error_value = None
    
    if row['isErrorWideR'] == 1:
        point_error_value = 'Wide Right'
    elif row['isErrorWideL'] == 1:
        point_error_value = 'Wide Left'
    elif 'isErrorNet' in row and row['isErrorNet'] == 1:
        point_error_value = 'Net'
    elif row['isErrorLong'] == 1:
        point_error_value = 'Long'
    

    # If an error is found, append the result to the error_results DataFrame
    if point_error_value is not None:
        error_results = pd.concat([error_results, pd.DataFrame({'pointNumber': [pointNumber], 'errorType': [point_error_value]})], ignore_index=True)


# Drop duplicates based on 'pointNumber'
error_results = error_results.drop_duplicates(subset=['pointNumber'])

In [83]:
# Create a dictionary mapping 'pointNumber' to 'errorType' in error_results
error_type_mapping = dict(zip(error_results['pointNumber'], error_results['errorType']))

# Create 'errorType' column in point_df based on the mapping
point_df['errorType'] = point_df['pointNumber'].map(error_type_mapping)

point_df = point_df.replace({np.nan: None})

### Add Column: Error (Returns)

In [84]:
def get_return_error(row):
    if row['rallyCount'] == 2:
        return row['errorType']
    else:
        return None

point_df.loc[point_df['pointNumber'] == point_number, 'serveInPlacement'] = shot_data['secondServeZone']  

# Apply the functions to create the new columns
point_df['returnError'] = point_df.apply(get_return_error, axis=1)


In [85]:
# print(point_df["returnError"].unique())
# print(point_df[['rallyCount','returnDirection','returnFhBh']])

### Add Column: Last Shot

In [86]:
point_df['lastShotDirection'] = None
point_df['lastShotFhBh'] = None
point_df['lastShotHitBy'] = None  
point_df['lastShotResult'] = None  

# Iterate through unique pointNumbers in shot_data
for point_number in shot_data['pointNumber'].unique():
    # Check if isPointEnd == 1 exists for the given pointNumber
    if 1 in shot_data.loc[shot_data['pointNumber'] == point_number, 'isPointEnd'].values:
        # Get the information from the corresponding row
        row_with_lastshot_info = shot_data[(shot_data['pointNumber'] == point_number) & (shot_data['isPointEnd'] == 1)].iloc[0]

        # Assign values to 'lastShotDirection' and 'lastShotFhBh' columns
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotDirection'] = row_with_lastshot_info['shotDirection']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotFhBh'] = row_with_lastshot_info['shotFhBh']
        point_df.loc[point_df['pointNumber'] == point_number, 'lastShotHitBy'] = row_with_lastshot_info['shotHitBy']
        
        # Determine lastShotResult based on conditions
        if row_with_lastshot_info['isWinner'] == 1 and not row_with_lastshot_info['isAce']:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Winner"
        elif row_with_lastshot_info['lastShotError'] == 1:
            point_df.loc[point_df['pointNumber'] == point_number, 'lastShotResult'] = "Error"

In [87]:
# # Display the modified DataFrame
# print(point_df[['rallyCount','lastShotDirection','lastShotFhBh', 'lastShotHitBy']])

In [88]:
print(point_df['lastShotResult'].unique())

['Error' None 'Winner']


### Add Column: pointWonBy

In [89]:
# Initialize variables to keep track of the state
prev_point_number = None
point_won_by_list = []

# Iterate through the DataFrame
for index, row in shot_data.iterrows():
    if row['isPointEnd'] == 1:
        # Check if pointNumber is different and consecutively increasing
        if prev_point_number is None or row['pointNumber'] == prev_point_number + 1:
            # Append pointWonBy to the list
            point_won_by_list.append(row['pointWonBy'])
            prev_point_number = row['pointNumber']
        else:
            print("Error: Point numbers are not different or consecutively increasing.")
            break

# Add point_won_by_list as a new column to point_df
point_df['pointWonBy'] = point_won_by_list

### Error Check: 
#### player1, player2, serverName, returnerName do not have misspellings

In [90]:
print(point_df['player1Name'].unique())
print(point_df['player2Name'].unique())
print(point_df['serverName'].unique())
print(point_df['returnerName'].unique())

['Tian Fangran']
['Irina Cantos Siemers']
['Irina Cantos Siemers' 'Tian Fangran']
['Tian Fangran' 'Irina Cantos Siemers']


### Error Check: 
#### pointWonBy has value that is not one of the two player names
- References: first_player1Name, first_player2Name from above server info in point_df

In [91]:
print(point_df['pointWonBy'].unique())
none_pointWonBy_df = point_df[~point_df['pointWonBy'].isin([first_player1Name, first_player2Name])]

print(none_pointWonBy_df)

['Tian Fangran' 'Irina Cantos Siemers']
Empty DataFrame
Columns: [pointNumber, player1Name, player2Name, pointScore, gameScore, setScore, tiebreakScore, side, serverName, returnerName, clientTeam, opponentTeam, Position, pointEndPosition, Duration, rallyCount, rallyCountFreq, firstServeIn, secondServeIn, serveResult, serveInPlacement, firstServeZone, secondServeZone, isAce, returnDirection, returnFhBh, errorType, returnError, lastShotDirection, lastShotFhBh, lastShotHitBy, lastShotResult, pointWonBy]
Index: []

[0 rows x 33 columns]


### Add Column: Break Point

In [92]:
# List of values to check for in 'Name' column
break_point_values = ['0-40', '15-40', '30-40', '40-40 (Deuce)', '40-40 (Ad)']

# Create 'isBreakPoint' column in point_df
point_df['isBreakPoint'] = point_df['pointScore'].isin(break_point_values)

## Warning: Do not use if atNetPlayer1 is in shot_data
### Add Column: atNetPlayer1, atNetPlayer2

- if below says atNetPlayer1: Uncomment below Code
- replace later with atNetPlayer1 atNetPlayer2

In [93]:
# Check if 'atNetPlayer1' exists in point_df
if 'atNetPlayer1' in shot_data.columns:
    print("Column 'atNetPlayer1' exists in point_df.")
else:
    print("Column 'atNetPlayer1' does not exist in point_df.")


if 'atNetPlayer2' in shot_data.columns:
    print("Column 'atNetPlayer2' exists in point_df.")
else:
    print("Column 'atNetPlayer2' does not exist in point_df.")

Column 'atNetPlayer1' exists in point_df.
Column 'atNetPlayer2' exists in point_df.


In [94]:
# Add firstServeIn and secondServeIn columns
point_df['atNetPlayer1'] = 0
point_df['atNetPlayer2'] = 0

for point_number in shot_data['pointNumber'].unique():
    # Check if firstServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['atNetPlayer1'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'atNetPlayer1'] = 1

    # Check if secondServeIn is 1 for the given pointNumber in shot_data
    if any((shot_data['pointNumber'] == point_number) & (shot_data['atNetPlayer2'] == 1)):
        point_df.loc[point_df['pointNumber'] == point_number, 'atNetPlayer2'] = 1

### Add Column: setNum

In [95]:
# Extract numbers from 'setScore' and calculate the sum plus 1
point_df['setNum'] = point_df['setScore'].apply(lambda x: sum(int(char) for char in x if char.isdigit()) + 1)

In [96]:
point_df.columns

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'setScore', 'tiebreakScore', 'side', 'serverName', 'returnerName',
       'clientTeam', 'opponentTeam', 'Position', 'pointEndPosition',
       'Duration', 'rallyCount', 'rallyCountFreq', 'firstServeIn',
       'secondServeIn', 'serveResult', 'serveInPlacement', 'firstServeZone',
       'secondServeZone', 'isAce', 'returnDirection', 'returnFhBh',
       'errorType', 'returnError', 'lastShotDirection', 'lastShotFhBh',
       'lastShotHitBy', 'lastShotResult', 'pointWonBy', 'isBreakPoint',
       'atNetPlayer1', 'atNetPlayer2', 'setNum'],
      dtype='object')

### Add Column: Game Number, Set Number, Game/Set/Point for each player

In [97]:
point_df[['player1SetScore', 'player2SetScore']] = point_df['setScore'].str.split('-', expand=True)
point_df[['player1GameScore', 'player2GameScore']] = point_df['gameScore'].str.split('-', expand=True)
point_df[['player1PointScore', 'player2PointScore']] = point_df['pointScore'].str.split('-', expand=True)
if not point_df['tiebreakScore'].isnull().all() and not point_df['tiebreakScore'].eq("").all():
    # Perform the operation only when tiebreakScore is not empty
    point_df[['player1TiebreakScore', 'player2TiebreakScore']] = point_df['tiebreakScore'].str.split('-', expand=True)
else:
    # Set player1TiebreakScore and player2TiebreakScore to NaN
    point_df['player1TiebreakScore'] = np.nan
    point_df['player2TiebreakScore'] = np.nan
    
def calculate_game_number(score):
    return int(score.split('-')[0]) + int(score.split('-')[1])+1

# Apply the function to create the 'gameNumber' column
point_df['gameNumber'] = point_df['gameScore'].apply(calculate_game_number)

### Error Check: 
#### Output Game Number in order: Should be consecutive increasing. Ex: 1,2,3,4,5,6. End of Set 1. 1,2,3,4,5,6,7,8

In [98]:
game_numbers = point_df['gameNumber'].tolist()

# Initialize variables
seen = set()
prev = None

# Iterate through gameNumber column
for num in game_numbers:
    # If the number is not in seen or it's different from the previous one, print it
    if num not in seen or num != prev:
        print(num, end=', ')
    # If the number is the same as the previous one but not consecutive, print it
    elif num == prev and num not in seen:
        print(num, end=', ')
    # Update seen set and prev variable
    seen.add(num)
    prev = num

1, 2, 3, 4, 5, 6, 7, 8, 9, 1, 2, 3, 4, 5, 6, 7, 8, 9, 

### Add Column: player1ServeResult

In [99]:
# Add the 'player1ServeResult' column
point_df['player1ServeResult'] = None

# Set player1ServeResult based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServeResult'] = point_df['serveResult']

In [100]:
point_df.loc[point_df['isAce'] == True, 'player1ServeResult'] = 'Ace'


In [101]:
# print(point_df['serveInPlacement'].head(10))

### Add Column: player1ServePlacement

In [102]:
# Add the 'player1ServePlacement' column
point_df['player1ServePlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['serverName'] == point_df['player1Name'], 'player1ServePlacement'] = point_df['side'] + ': ' + point_df['serveInPlacement']

### Add Column: player1ReturnPlacement

In [103]:
# Add the 'player1ReturnPlacement' column
point_df['player1ReturnPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnPlacement'] = point_df['returnDirection']

### Add Column: player1ReturnFhBh

In [104]:
# Add the 'player1ReturnFhBh' column
point_df['player1ReturnFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player1Name'], 'player1ReturnFhBh'] = point_df['returnFhBh']

### Add Column: player1LastShotPlacement

In [105]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotPlacement'] = point_df['lastShotDirection']

### Add Column: player1LastShotFhBh

In [106]:
# Add the 'player1LastShotFhBh' column
point_df['player1LastShotFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player1Name'], 'player1LastShotFhBh'] = point_df['lastShotFhBh']

### Add Column: player1LastShotResult

In [107]:
# Add the 'player1LastShotResult' column
point_df['player1LastShotResult'] = None

# Set player1LastShotResult based on conditions, excluding 'Ace' and 'Double Fault'
point_df.loc[
    (point_df['lastShotHitBy'] == point_df['player1Name']) & 
    ~point_df['player1ServeResult'].isin(['Ace', 'Double Fault']), 
    'player1LastShotResult'
] = point_df['lastShotResult']


In [108]:
## Add Column: player2ServeResult

In [109]:
# Add the 'player2ServeResult' column
point_df['player2ServeResult'] = None

# Set player1ServeResult based on conditions
point_df.loc[point_df['serverName'] == point_df['player2Name'], 'player2ServeResult'] = point_df['serveResult']

In [110]:
point_df.loc[point_df['isAce'] == True, 'player1ServeResult'] = 'Ace'


In [111]:
# print(point_df['serveInPlacement'].head(10))

### Add Column: player2ServePlacement

In [112]:
# Add the 'player1ServePlacement' column
point_df['player2ServePlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['serverName'] == point_df['player2Name'], 'player2ServePlacement'] = point_df['side'] + ': ' + point_df['serveInPlacement']

## Add Column: player2ReturnPlacement

In [113]:
# Add the 'player2ReturnPlacement' column
point_df['player2ReturnPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player2Name'], 'player2ReturnPlacement'] = point_df['returnDirection']

### Add Column: player2ReturnFhBh

In [114]:
# Add the 'player1ReturnFhBh' column
point_df['player2ReturnFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['returnerName'] == point_df['player2Name'], 'player2ReturnFhBh'] = point_df['returnFhBh']

### Add Column: player2LastShotPlacement

In [115]:
# Add the 'player1LastShotFhBh' column
point_df['player2LastShotPlacement'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player2Name'], 'player2LastShotPlacement'] = point_df['lastShotDirection']

### Add Column: player2LastShotFhBh

In [116]:
# Add the 'player1LastShotFhBh' column
point_df['player2LastShotFhBh'] = None

# Set player1ServePlacement based on conditions
point_df.loc[point_df['lastShotHitBy'] == point_df['player2Name'], 'player2LastShotFhBh'] = point_df['lastShotFhBh']

## Add Column: player2LastShotResult

In [117]:
# Add the 'player2LastShotResult' column
point_df['player2LastShotResult'] = None

# Set player1LastShotResult based on conditions, excluding 'Ace' and 'Double Fault'
point_df.loc[
    (point_df['lastShotHitBy'] == point_df['player2Name']) &
    ~point_df['player2ServeResult'].isin(['Ace', 'Double Fault']),
    'player2LastShotResult'
] = point_df['lastShotResult']


In [118]:
print(point_df[['setNum', 'gameScore','pointScore','player1ServePlacement','serveInPlacement']].head(10))

    setNum gameScore pointScore player1ServePlacement serveInPlacement
0        1       0-0        0-0                  None                T
5        1       0-0       0-15                  None             None
6        1       0-0       0-30                  None                T
11       1       0-0      15-30                  None             Wide
15       1       0-0      30-30                  None                T
17       1       0-0      40-30                  None             Wide
20       1       0-0      40-40                  None             Wide
30       1       1-0        0-0                   NaN             None
31       1       1-0       0-15              Ad: Wide             Wide
33       1       1-0       0-30           Deuce: Wide             Wide


In [119]:
print(point_df[['setNum', 'gameScore','pointScore','player1ServePlacement','serveInPlacement']].head(10))

    setNum gameScore pointScore player1ServePlacement serveInPlacement
0        1       0-0        0-0                  None                T
5        1       0-0       0-15                  None             None
6        1       0-0       0-30                  None                T
11       1       0-0      15-30                  None             Wide
15       1       0-0      30-30                  None                T
17       1       0-0      40-30                  None             Wide
20       1       0-0      40-40                  None             Wide
30       1       1-0        0-0                   NaN             None
31       1       1-0       0-15              Ad: Wide             Wide
33       1       1-0       0-30           Deuce: Wide             Wide


## Add Column: Name (For Video)

In [120]:
# Change pointScore to the specified format
point_df['Name'] = point_df.apply(lambda row: f"Set {row['setNum']}: {row['gameScore']}, {row['tiebreakScore']} {row['serverName']} Serving" if pd.notna(row['tiebreakScore']) else f"Set {row['setNum']}: {row['gameScore']}, {row['pointScore']} {row['serverName']} Serving", axis=1)


In [121]:
print(point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']].head(5))
print(point_df[['setNum', 'gameScore','pointScore','tiebreakScore','Name']].tail(5))

    setNum gameScore pointScore tiebreakScore  \
0        1       0-0        0-0          None   
5        1       0-0       0-15          None   
6        1       0-0       0-30          None   
11       1       0-0      15-30          None   
15       1       0-0      30-30          None   

                                              Name  
0     Set 1: 0-0, 0-0 Irina Cantos Siemers Serving  
5    Set 1: 0-0, 0-15 Irina Cantos Siemers Serving  
6    Set 1: 0-0, 0-30 Irina Cantos Siemers Serving  
11  Set 1: 0-0, 15-30 Irina Cantos Siemers Serving  
15  Set 1: 0-0, 30-30 Irina Cantos Siemers Serving  
     setNum gameScore pointScore tiebreakScore  \
540       2       3-5        0-0          None   
547       2       3-5       0-15          None   
564       2       3-5       0-30          None   
569       2       3-5      15-30          None   
575       2       3-5      15-40          None   

                                       Name  
540    Set 2: 3-5, 0-0 Tian Fangran Serv

## Reorder DataFrame for Output

In [122]:
point_df_copy = point_df.copy()

In [123]:
point_df.shape

(107, 61)

In [124]:
point_df_copy.shape

(107, 61)

In [125]:
point_df.columns

Index(['pointNumber', 'player1Name', 'player2Name', 'pointScore', 'gameScore',
       'setScore', 'tiebreakScore', 'side', 'serverName', 'returnerName',
       'clientTeam', 'opponentTeam', 'Position', 'pointEndPosition',
       'Duration', 'rallyCount', 'rallyCountFreq', 'firstServeIn',
       'secondServeIn', 'serveResult', 'serveInPlacement', 'firstServeZone',
       'secondServeZone', 'isAce', 'returnDirection', 'returnFhBh',
       'errorType', 'returnError', 'lastShotDirection', 'lastShotFhBh',
       'lastShotHitBy', 'lastShotResult', 'pointWonBy', 'isBreakPoint',
       'atNetPlayer1', 'atNetPlayer2', 'setNum', 'player1SetScore',
       'player2SetScore', 'player1GameScore', 'player2GameScore',
       'player1PointScore', 'player2PointScore', 'player1TiebreakScore',
       'player2TiebreakScore', 'gameNumber', 'player1ServeResult',
       'player1ServePlacement', 'player1ReturnPlacement', 'player1ReturnFhBh',
       'player1LastShotPlacement', 'player1LastShotFhBh',
       'pla

In [126]:
desired_order = [
    'Name', 'pointNumber', 'setNum', 'gameNumber', 'player1Name', 'player2Name',
    'pointScore', 'gameScore', 'setScore', 'tiebreakScore', 'side', 'serverName',
    'returnerName', 'Position', 'pointEndPosition', 'Duration', 'pointWonBy',
    'rallyCount', 'rallyCountFreq', 'firstServeIn', 'secondServeIn',
    'serveResult', 'serveInPlacement', 'firstServeZone', 'secondServeZone',
    'isAce', 'returnDirection', 'returnFhBh', 'errorType', 'returnError',
    'lastShotDirection', 'lastShotFhBh', 'lastShotHitBy', 'lastShotResult',
    'isBreakPoint', 'atNetPlayer1', 'atNetPlayer2', 'player1SetScore',
    'player2SetScore', 'player1GameScore', 'player2GameScore', 'player1PointScore',
    'player2PointScore', 'player1TiebreakScore', 'player2TiebreakScore',
    'player1ServeResult', 'player1ServePlacement', 'player1ReturnPlacement',
    'player1ReturnFhBh', 'player1LastShotPlacement', 'player1LastShotFhBh',
    'player1LastShotResult', 'player2ServeResult', 'player2ServePlacement',
       'player2ReturnPlacement', 'player2ReturnFhBh',
       'player2LastShotPlacement', 'player2LastShotFhBh',
       'player2LastShotResult', 'clientTeam', 'opponentTeam'
]

# Reorder the columns
point_df = point_df.reindex(columns=desired_order)

### Error Check: 
#### Check if the columns and their order are the same

In [127]:
print(point_df.shape)
print(point_df_copy.shape)

(107, 61)
(107, 61)


In [128]:
# Get the set of column names for each DataFrame
point_df_columns = set(point_df.columns)
point_df_copy_columns = set(point_df_copy.columns)

# Find the column names unique to each DataFrame
unique_to_point_df = point_df_columns - point_df_copy_columns
unique_to_point_df_copy = point_df_copy_columns - point_df_columns

# Output the results
if unique_to_point_df:
    print("Columns unique to point_df:", unique_to_point_df)
else:
    print("All columns in point_df are also in point_df_copy")

if unique_to_point_df_copy:
    print("Columns unique to point_df_copy:", unique_to_point_df_copy)
else:
    print("All columns in point_df_copy are also in point_df")


All columns in point_df are also in point_df_copy
All columns in point_df_copy are also in point_df


## Warning: All nan to ""
### Alterating Data

In [129]:
# Change Na to "" Empty String

# Assuming point_df is your DataFrame
point_df.replace([pd.NA, None, pd.NaT, float('nan')], "", inplace=True)

# Double Check before Outputting Point (FOR VISUAL)

In [130]:
print(point_df.head(5))

                                              Name  pointNumber  setNum  \
0     Set 1: 0-0, 0-0 Irina Cantos Siemers Serving            1       1   
5    Set 1: 0-0, 0-15 Irina Cantos Siemers Serving            2       1   
6    Set 1: 0-0, 0-30 Irina Cantos Siemers Serving            3       1   
11  Set 1: 0-0, 15-30 Irina Cantos Siemers Serving            4       1   
15  Set 1: 0-0, 30-30 Irina Cantos Siemers Serving            5       1   

    gameNumber   player1Name           player2Name pointScore gameScore  \
0            1  Tian Fangran  Irina Cantos Siemers        0-0       0-0   
5            1  Tian Fangran  Irina Cantos Siemers       0-15       0-0   
6            1  Tian Fangran  Irina Cantos Siemers       0-30       0-0   
11           1  Tian Fangran  Irina Cantos Siemers      15-30       0-0   
15           1  Tian Fangran  Irina Cantos Siemers      30-30       0-0   

   setScore tiebreakScore  ... player1LastShotResult player2ServeResult  \
0       0-0            

In [131]:
print(point_df.tail(5))

                                       Name  pointNumber  setNum  gameNumber  \
540    Set 2: 3-5, 0-0 Tian Fangran Serving          103       2           9   
547   Set 2: 3-5, 0-15 Tian Fangran Serving          104       2           9   
564   Set 2: 3-5, 0-30 Tian Fangran Serving          105       2           9   
569  Set 2: 3-5, 15-30 Tian Fangran Serving          106       2           9   
575  Set 2: 3-5, 15-40 Tian Fangran Serving          107       2           9   

      player1Name           player2Name pointScore gameScore setScore  \
540  Tian Fangran  Irina Cantos Siemers        0-0       3-5      0-1   
547  Tian Fangran  Irina Cantos Siemers       0-15       3-5      0-1   
564  Tian Fangran  Irina Cantos Siemers       0-30       3-5      0-1   
569  Tian Fangran  Irina Cantos Siemers      15-30       3-5      0-1   
575  Tian Fangran  Irina Cantos Siemers      15-40       3-5      0-1   

    tiebreakScore  ... player1LastShotResult player2ServeResult  \
540          

In [167]:
# Change Value Names for Match Viewer Output

# Assuming point_df is your DataFrame and player1Name is the name from the first row
player1Name = point_df.iloc[0]['player1Name']
player2Name = point_df.iloc[0]['player2Name']

# Replace values in the 'atNetPlayer1' column
point_df['atNetPlayer1'] = point_df['atNetPlayer1'].replace({0: "", 1: player1Name})
point_df['atNetPlayer2'] = point_df['atNetPlayer2'].replace({0: "", 1: player2Name})

# Add in Timestamps

In [168]:
timestamp = pd.read_csv("Tian v. Ohio Timestamps - Sheet1.csv")

In [169]:
# Error Check, timestamp size should be same as point_df size

In [170]:
if timestamp.shape[0] != point_df.shape[0]:
    print("Error: The number of rows in timestamp and point_df are not the same.")
else:
    # Assign values to point_df
    point_df['Position'] = timestamp['pointStartTime'].values
    point_df['pointEndPosition'] = timestamp['pointEndTime'].values


# Check that Time Stamp head and end is same as point_df

In [171]:
print(timestamp.head())

   pointStartTime  pointEndTime
0            2968         11195
1           32766         45638
2           73331         84832
3          104089        123298
4          144779        149950


In [172]:
print(timestamp.tail())

     pointStartTime  pointEndTime
102         5292198       5305858
103         5328655       5370900
104         5418691       5427429
105         5468823       5480818
106         5516354       5534543


In [173]:
print(point_df[['Position', 'pointEndPosition']])

     Position  pointEndPosition
0        2968             11195
5       32766             45638
6       73331             84832
11     104089            123298
15     144779            149950
..        ...               ...
540   5292198           5305858
547   5328655           5370900
564   5418691           5427429
569   5468823           5480818
575   5516354           5534543

[107 rows x 2 columns]


# PRINT POINTCSV VISUALS

In [174]:
# Save point_df to CSV file

# Assuming point_df is your DataFrame and player1Name and player2Name are the names from the first row
player1NameNoSpace = point_df.iloc[0]['player1Name'].replace(" ", "")
player2NameNoSpace = point_df.iloc[0]['player2Name'].replace(" ", "")

# Save DataFrame to CSV file with modified player names
point_df.to_csv(f'Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv', index=False)


In [176]:



def csv_to_json(csv_file_path, json_file_name):
    # Load CSV file into a pandas DataFrame
    df = pd.read_csv(csv_file_path)
    
    # Convert DataFrame to JSON format
    json_data = df.to_json(orient='records')
    
    # Save JSON data to a file in the same directory
    json_file_path = f'{json_file_name}'
    with open(json_file_path, 'w') as json_file:
        json_file.write(json_data)
    
    return json_file_path

# Define the CSV file path
csv_file_path = f"Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.csv"

# Define the desired JSON file name (without extension)
json_file_name = f"Point_Visuals_{player1NameNoSpace}_{player2NameNoSpace}.json"

# Convert CSV to JSON and save in the same directory
output_file_path = csv_to_json(csv_file_path, json_file_name)

# Display the path where the JSON file is saved
output_file_path


'Point_Visuals_TianFangran_IrinaCantosSiemers.json'

# EDA

## Shot CSV EDA

In [ ]:
# Can input CSV Directly here for statistics functions

# your_file_name = "filename.csv"
# shot_eda = pd.read_csv(your_file_name)

# if directly from notebook
shot_eda = shot_data.copy()

In [ ]:
player1Name = shot_eda.iloc[0]['player1Name']

# Filter shot_data based on the conditions
approach_data_player1 = shot_eda[(shot_eda['isApproach'] == 1) & (shot_eda['shotHitBy'] == player1Name)]

# Count the distinct pointNumbers
distinct_point_numbers = approach_data_player1['pointNumber'].nunique()

# Print the result
print(f"Number of Approach Shots hit by {player1Name}: {distinct_point_numbers}" )

# print(approach_data_player1)


In [ ]:
print(point_df['Name'])

## Point CSV EDA

In [ ]:
# Can input CSV Directly here for statistics functions
import pandas as pd
your_file_name = "Point_Visuals_Anne-ChristineLutkemeyer_SehaYu.csv"
point_df_eda = pd.read_csv(your_file_name)

# OR MAKE point_df_eda copy of point_df

# point_df_eda = point_df.copy()

In [ ]:
print(point_df_eda.columns)

In [ ]:

serve_in_rows = point_df_eda[(point_df_eda['serveResult'] == '1st Serve In') | (point_df_eda['serveResult'] == '2nd Serve In')]

# filter the rows where rallyCount column is equal to 1
desired_rows = serve_in_rows[serve_in_rows['rallyCount'] == 1]

# print each row
for index, row in desired_rows.iterrows():
    print(row[['pointNumber', 'setNum', 'gameNumber','serveResult','rallyCount' ,'serverName']])


In [ ]:
first_player1Name = point_df_eda['player1Name'].iloc[0]



# Display the results
print(f"\nServe Results for {first_player1Name}:")

# Assuming point_df_eda is your DataFrame
total_serves = len(point_df_eda[point_df_eda['serverName'] == first_player1Name])
first_serve_in_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 1)])
first_serve_won_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 1) & (point_df_eda['pointWonBy'] == first_player1Name)])
percentage_first_serve_in = (first_serve_in_count / total_serves) * 100 if total_serves > 0 else 0
percentage_first_serve_won = (first_serve_won_count / first_serve_in_count) * 100 if first_serve_in_count > 0 else 0

second_serve_total_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 0)])
second_serve_in_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 0)& (point_df_eda['secondServeIn'] == 1)])
second_serve_won_count = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['firstServeIn'] == 0)& (point_df_eda['secondServeIn'] == 1) & (point_df_eda['pointWonBy'] == first_player1Name)])
percentage_second_serve_in = (second_serve_in_count / second_serve_total_count) * 100 if second_serve_total_count > 0 else 0
percentage_second_serve_won = (second_serve_won_count / second_serve_in_count) * 100 if second_serve_in_count > 0 else 0



# Display the results
print("\nTotal Serves:", total_serves)
print("First Serve In (Count):", first_serve_in_count)
print("First Serve Won (Count):", first_serve_won_count)
print(f"First Serve In (%): {percentage_first_serve_in:.2f}%")
print(f"First Serve Won (%): {percentage_first_serve_won:.2f}%")

print("Second Serve In (Count):", second_serve_in_count)
print("Second Serve Total (Count):", second_serve_total_count)
print("Second Serve Won (Count):", second_serve_won_count)
print(f"Second Serve In (%): {percentage_second_serve_in:.2f}%")
print(f"Second Serve Won (%): {percentage_second_serve_won:.2f}%")

# Assuming point_df is your DataFrame
count_is_ace = (point_df_eda[point_df_eda['serverName'] == first_player1Name]['isAce']).sum()
count_is_double_fault = ((point_df_eda['serverName'] == first_player1Name) & (point_df_eda['serveResult'] == "Double Fault")).sum()

# Display the results
print("Ace (Count):", count_is_ace)
print("Double Fault (Count):", count_is_double_fault)

# Count of rows where serverName is equal to the first row of player1Name and pointWonBy is equal to the first row of player1Name
total_service_points_won = len(point_df_eda[(point_df_eda['serverName'] == first_player1Name) & (point_df_eda['pointWonBy'] == first_player1Name)])
total_service_points_won_percentage = total_service_points_won / total_serves *100

# Display the results
print(f"Points Won on Serve (Count) {total_service_points_won}")

print(f"Points Won on Serve (%): {total_service_points_won_percentage:.2f}%")

# Assuming point_df is your DataFrame
return_points = point_df_eda[(point_df_eda['returnerName'] == first_player1Name)] # CHANGED THIS JERRY, REMOVED RALLY COUNT >= 2

total_return = len(return_points)
returnMade = len(return_points[(return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
returnError = len(return_points[(return_points['lastShotResult'] == 'Error') & (return_points['rallyCount'] == 2)])
returnWinner = len(return_points[(return_points['lastShotResult'] == 'Winner') & (return_points['rallyCount'] == 2)])
returnMadePercentage = returnMade/total_return

returnWonByPlayer1 = len(return_points[return_points['pointWonBy'] == first_player1Name])
returnWonByPlayer1Percentage = returnWonByPlayer1 / total_return * 100 if total_return > 0 else 0

deuceReturnCount = len(return_points[return_points['side'] == 'Deuce'])
adReturnCount = len(return_points[return_points['side'] == 'Ad'])


deuceReturnMade = len(return_points[(return_points['side'] == 'Deuce') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])
adReturnMade = len(return_points[(return_points['side'] == 'Ad') & ((return_points['rallyCount'] > 2) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error')))])

deuceReturnMadePercentage = deuceReturnMade/deuceReturnCount
adReturnMadePercentage = adReturnMade/adReturnCount

deuceReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Deuce') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])
adReturnWonByPlayer1 = len(return_points[(return_points['side'] == 'Ad') & (return_points['pointWonBy'] == first_player1Name) | ((return_points['rallyCount'] == 2) & (return_points['lastShotResult'] != 'Error'))])

deuceReturnWonByPlayer1Percentage = deuceReturnWonByPlayer1 / deuceReturnCount * 100 if deuceReturnCount > 0 else 0
adReturnWonByPlayer1Percentage = adReturnWonByPlayer1 / adReturnCount * 100 if adReturnCount > 0 else 0




print(f"\nReturn Results for {first_player1Name}:\n")

print("Total Return (Count):", total_return)
print("Return Won (Count):", returnWonByPlayer1)
print("Return Won (%):", returnWonByPlayer1Percentage)

print("\nReturn Made (Count):", returnMade)
print("Return Made (%):", returnMadePercentage)
print("Return Error (Count):", returnError)
print("Return Winner (Count):", returnWinner)

print("\nDeuce Return (Count):", deuceReturnCount)
print("Deuce Return Made (Count):", deuceReturnMade)
print("Deuce Return Made (%):", deuceReturnMadePercentage)
print("Deuce Return Won by Player1 (%):", deuceReturnWonByPlayer1Percentage)
print("Deuce Return Won by Player1 (Count):", deuceReturnWonByPlayer1)


print("\nAd Return (Count):", adReturnCount)
print("Ad Return Made (Count):", adReturnMade)
print("Ad Return Made (%):", adReturnMadePercentage)
print("Ad Return Won by Player1 (Count):", adReturnWonByPlayer1)
print("Ad Return Won by Player1 (%):", adReturnWonByPlayer1Percentage)

# Assuming return_points is your DataFrame
deuce_return_points = return_points[(return_points['side'] == 'Deuce') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Deuce Return Points Separated by returnFhBh
deuce_forehand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Forehand']
deuce_backhand_return_points = deuce_return_points[deuce_return_points['returnFhBh'] == 'Backhand']


# Count for Deuce Return Points - Made
count_deuce_forehand_made = len(deuce_forehand_return_points[(deuce_forehand_return_points['rallyCount'] > 2) | ((deuce_forehand_return_points['rallyCount'] == 2) & (deuce_forehand_return_points['lastShotResult'] != 'Error'))])
count_deuce_backhand_made = len(deuce_backhand_return_points[(deuce_backhand_return_points['rallyCount'] > 2) | ((deuce_backhand_return_points['rallyCount'] == 2) & (deuce_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Deuce Return Points - Error
count_deuce_forehand_error = len(deuce_forehand_return_points[(deuce_forehand_return_points['lastShotResult'] == 'Error') & (deuce_forehand_return_points['rallyCount'] == 2)])
count_deuce_backhand_error = len(deuce_backhand_return_points[(deuce_backhand_return_points['lastShotResult'] == 'Error') & (deuce_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("\nDeuce Forehand Return Points - Made:", count_deuce_forehand_made)
print("Deuce Forehand Return Points - Error:", count_deuce_forehand_error)

print("Deuce Backhand Return Points - Made:", count_deuce_backhand_made)
print("Deuce Backhand Return Points - Error:", count_deuce_backhand_error)

# Assuming return_points is your DataFrame
ad_return_points = return_points[(return_points['side'] == 'Ad') & (return_points['returnerName'] == first_player1Name) & (return_points['rallyCount'] >= 2)]

# Ad Return Points Separated by returnFhBh
ad_forehand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Forehand']
ad_backhand_return_points = ad_return_points[ad_return_points['returnFhBh'] == 'Backhand']

# Count for Ad Return Points - Made
count_ad_forehand_made = len(ad_forehand_return_points[(ad_forehand_return_points['rallyCount'] > 2) | ((ad_forehand_return_points['rallyCount'] == 2) & (ad_forehand_return_points['lastShotResult'] != 'Error'))])
count_ad_backhand_made = len(ad_backhand_return_points[(ad_backhand_return_points['rallyCount'] > 2) | ((ad_backhand_return_points['rallyCount'] == 2) & (ad_backhand_return_points['lastShotResult'] != 'Error'))])

# Count for Ad Return Points - Error
count_ad_forehand_error = len(ad_forehand_return_points[(ad_forehand_return_points['lastShotResult'] == 'Error') & (ad_forehand_return_points['rallyCount'] == 2)])
count_ad_backhand_error = len(ad_backhand_return_points[(ad_backhand_return_points['lastShotResult'] == 'Error') & (ad_backhand_return_points['rallyCount'] == 2)])

# Display the counts
print("\nAd Forehand Return Points - Made:", count_ad_forehand_made)
print("Ad Forehand Return Points - Error:", count_ad_forehand_error)

print("Ad Backhand Return Points - Made:", count_ad_backhand_made)
print("Ad Backhand Return Points - Error:", count_ad_backhand_error)

print(f"\nAt Net Results for {first_player1Name}:\n")


# Total points where atNetPlayer1 = 1
total_at_net_player1 = len(point_df_eda[point_df_eda['atNetPlayer1'] == 1])

# Percentage of points where atNetPlayer1 = 1 out of total points
percentage_at_net_player1 = (total_at_net_player1 / len(point_df_eda)) * 100 if len(point_df_eda) > 0 else 0

# Display the total count and percentage of points where atNetPlayer1 = 1
print(f"Total Net Points for {first_player1Name}: {total_at_net_player1}")
print(f"Percentage of Net Points for {first_player1Name}: {percentage_at_net_player1:.2f}%")

# Points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
at_net_player1_and_won_by_player1 = len(point_df_eda[(point_df_eda['atNetPlayer1'] == 1) & (point_df_eda['pointWonBy'] == first_player1Name)])

# Percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name out of total points where atNetPlayer1 = 1
percentage_at_net_player1_and_won_by_player1 = (at_net_player1_and_won_by_player1 / total_at_net_player1) * 100 if total_at_net_player1 > 0 else 0

# Display the count and percentage of points where atNetPlayer1 = 1 and pointWonBy = first_player1Name
print(f"\nTotal Net Points won by {first_player1Name}: {at_net_player1_and_won_by_player1}")
print(f"Percentage of Net Points won by {first_player1Name}: {percentage_at_net_player1_and_won_by_player1:.2f}%")


In [ ]:
# Filter points where serverName is equal to first_player1Name
filtered_points = point_df_eda[point_df_eda['serverName'] == first_player1Name]

# Group the filtered points by player1ServePlacement and count the occurrences
serve_placement_counts = filtered_points.groupby('player1ServePlacement').size()


# Iterate over filtered_points
for index, point in filtered_points.iterrows():
    serve_placement = point['player1ServePlacement']
    
    # Check if serve placement is not in serve_placement_counts
    if serve_placement not in serve_placement_counts:
        print(point['Name'])


# Initialize dictionaries to store counts and percentages
point_won_counts = {}
point_won_percentages = {}
print(f"Total {len(filtered_points)}")

# Iterate over serve placements
for serve_placement, count in serve_placement_counts.items():
    # Filter points with the specific serve placement
    serve_placement_points = filtered_points[filtered_points['player1ServePlacement'] == serve_placement]
    
    # Count points won by first_player1Name
    point_won_count = serve_placement_points[serve_placement_points['pointWonBy'] == first_player1Name].shape[0]
    
    # Calculate percentage
    point_won_percentage = (point_won_count / count) * 100 if count > 0 else 0
    
    # Store counts and percentages
    point_won_counts[serve_placement] = point_won_count
    point_won_percentages[serve_placement] = point_won_percentage

# Print counts and percentages
for serve_placement, count in serve_placement_counts.items():
    print(f"Serve Placement: {serve_placement}")
    print(f"Total Serves: {count}")
    print(f"Serves Won by {first_player1Name}: {point_won_counts.get(serve_placement, 0)}")
    print(f"Percentage: {point_won_percentages.get(serve_placement, 0):.2f}%\n")

In [ ]:
print(f"\nError Data for {first_player1Name}:\n")
# Filter the DataFrame based on specified conditions
total_errors = point_df_eda[(point_df_eda['lastShotHitBy'] == first_player1Name) & 
                           (point_df_eda['lastShotResult'] == 'Error')]

import numpy as np

# Filter rows without NaN values in relevant columns
forehand_errors = point_df_eda[(point_df_eda['lastShotHitBy'] == first_player1Name) & 
                           (point_df_eda['lastShotResult'] == 'Error') &
                           (point_df_eda['lastShotFhBh'] == 'Forehand') &
                           (~point_df_eda['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

backhand_errors = point_df_eda[(point_df_eda['lastShotHitBy'] == first_player1Name) & 
                           (point_df_eda['lastShotResult'] == 'Error') &
                           (point_df_eda['lastShotFhBh'] == 'Backhand') &
                           (~point_df_eda['errorType'].isnull())]  # Ensure 'errorType' column doesn't have NaN

# Count the occurrences of 'Forehand' and 'Backhand' separately
forehand_counts = forehand_errors.shape[0]  # Count rows
backhand_counts = backhand_errors.shape[0]  # Count rows

# Print the total error counts for verification
total_error_counts = forehand_counts + backhand_counts

# Get value counts of 'errorType' for Forehand errors
forehand_error_types = forehand_errors['errorType'].value_counts(dropna=False)  # Include NaN values in count

# Get value counts of 'errorType' for Backhand errors
backhand_error_types = backhand_errors['errorType'].value_counts(dropna=False)  # Include NaN values in count

# Print the counts and error types
print("Count of Total errors:", total_error_counts)
print("Count of Forehand errors:", forehand_counts)
print("Forehand Error %:", forehand_counts/total_error_counts)
print("Count of Backhand errors:", backhand_counts)
print("Backhand Error %:", backhand_counts/total_error_counts)
print("\nForehand errors:\n", forehand_error_types)


# # Group by both 'lastShotDirection' and 'errorType', and then count occurrences
forehand_error_counts = forehand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0

print("\nValue counts of 'errorType' for Forehand errors with different directions:\n", forehand_error_counts)


print("\nBackhand errors:\n", backhand_error_types)



# Group by both 'lastShotDirection' and 'errorType', and then count occurrences
backhand_error_counts = backhand_errors.groupby(['player1LastShotPlacement', 'errorType']).size().unstack(fill_value=0)  # Fill NaN with 0

print("\nValue counts of 'errorType' for Backhand errors with different directions:\n", backhand_error_counts)

In [ ]:
pip install reportlab

In [ ]:
from reportlab.lib import colors
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle

# Your raw output data
raw_data = [
    {"Serve Placement": "Ad: Body", "Total Serves": 3, "Serves Won by Tian Fang Ran": 3, "Percentage": "100.00%"},
    {"Serve Placement": "Ad: T", "Total Serves": 11, "Serves Won by Tian Fang Ran": 8, "Percentage": "72.73%"},
    {"Serve Placement": "Ad: Wide", "Total Serves": 19, "Serves Won by Tian Fang Ran": 10, "Percentage": "52.63%"},
    {"Serve Placement": "Deuce: Body", "Total Serves": 5, "Serves Won by Tian Fang Ran": 2, "Percentage": "40.00%"},
    {"Serve Placement": "Deuce: T", "Total Serves": 19, "Serves Won by Tian Fang Ran": 10, "Percentage": "52.63%"},
    {"Serve Placement": "Deuce: Wide", "Total Serves": 14, "Serves Won by Tian Fang Ran": 7, "Percentage": "50.00%"}
]

# Convert raw data to list of lists for table
table_data = [["Serve Placement", "Total Serves", "Serves Won by Tian Fang Ran", "Percentage"]]
for row in raw_data:
    table_data.append([row["Serve Placement"], row["Total Serves"], row["Serves Won by Tian Fang Ran"], row["Percentage"]])

# Create PDF
pdf_filename = "tennis_stats.pdf"
doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
table = Table(table_data)

# Add style to table
style = TableStyle([('BACKGROUND', (0, 0), (-1, 0), colors.gray),
                    ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
                    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
                    ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
                    ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
                    ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
                    ('GRID', (0, 0), (-1, -1), 1, colors.black)])
table.setStyle(style)

# Add table to PDF
doc.build([table])
